# Using Attention for Neural Machine Translation
In this notebook we are going to perform machine translation using a deep learning based approach and attention mechanism.

Specifically, we are going to train a sequence to sequence model for Spanish to English translation.  We will use Sequence to Sequence Models for this Assignment. In this assignment you only need tto implement the encoder and decoder, we implement all the data loading for you.Please **refer** to the following resources for more details:

1.   https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
2.   https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
3. https://arxiv.org/pdf/1409.0473.pdf



In [1]:
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import unicodedata
import re
import time
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
print(torch.__version__)

1.3.0+cu100


# Download The Data

Here we will download the translation data. We will learn a model to translate Spanish to English.

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
cd sample_data/

In [0]:
!wget http://www.manythings.org/anki/spa-eng.zip

In [0]:
!unzip spa-eng.zip

In [0]:
f = open('spa.txt', encoding='UTF-8').read().strip().split('\n')
lines = f
total_num_examples = 30000 
original_word_pairs = [[w for w in l.split('\t')][:2] for l in lines[:total_num_examples]]
data = pd.DataFrame(original_word_pairs, columns=["eng", "es"])
data # visualizing the data

In [0]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    """
    Normalizes latin chars with accent to their canonical decomposition
    """
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

# Preprocessing the sentence to add the start, end tokens and make them lower-case
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    w = '<start> ' + w + ' <end>'
    return w

In [0]:
# Now we do the preprocessing using pandas and lambdas
# Make sure YOU only run this once - if you run it twice it will mess up the data so you will have run the few above cells again
data["eng"] = data.eng.apply(lambda w: preprocess_sentence(w))
data["es"] = data.es.apply(lambda w: preprocess_sentence(w))
data[250:260]

# Vocabulary Class

We create a class here for managing our vocabulary as we did in MP2. In this MP, we have a separate class for the vocabulary as we need 2 different vocabularies - one for English and one for Spanish.

In [0]:
class Vocab_Lang():
    def __init__(self, data):
        """ data is the list of all sentences in the language dataset"""
        self.data = data
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        
        self.create_index()
        
    def create_index(self):
        for sentence in self.data:
            # update with individual tokens
            self.vocab.update(sentence.split(' '))

        # add a padding token
        self.word2idx['<pad>'] = 0
        
        # word to index mapping
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1 # +1 because of pad token
        
        # index to word mapping
        for word, index in self.word2idx.items():
            self.idx2word[index] = word 

In [0]:
# index language using the class above
inp_lang = Vocab_Lang(data["es"].values.tolist())
targ_lang = Vocab_Lang(data["eng"].values.tolist())
# Vectorize the input and target languages
input_tensor = [[inp_lang.word2idx[s] for s in es.split(' ')]  for es in data["es"].values.tolist()]
target_tensor = [[targ_lang.word2idx[s] for s in eng.split(' ')]  for eng in data["eng"].values.tolist()]

In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
# calculate the max_length of input and output tensor for padding
max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)

In [0]:
def pad_sequences(x, max_len):
    padded = np.zeros((max_len), dtype=np.int64)
    if len(x) > max_len: padded[:] = x[:max_len]
    else: padded[:len(x)] = x
    return padded

# pad all the sentences in the dataset with the max_length
input_tensor = [pad_sequences(x, max_length_inp) for x in input_tensor]
target_tensor = [pad_sequences(x, max_length_tar) for x in target_tensor]

In [0]:
# Creating training and test/val sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = input_tensor[:24000], input_tensor[24000:], target_tensor[:24000], target_tensor[24000:]

assert(len(input_tensor_train)==24000)
assert(len(target_tensor_train)==24000)
assert(len(input_tensor_val)==6000)
assert(len(target_tensor_val)==6000)

# Dataloader for our Encoder and Decoder

We prepare the dataloader and make sure the dataloader returns the source sentence, target sentence and the length of the source sentenc sampled from the training dataset.

In [0]:
# conver the data to tensors and pass to the Dataloader 
# to create an batch iterator
from torch.utils.data import Dataset, DataLoader
class MyData(Dataset):
    def __init__(self, X, y):
        self.data = X
        self.target = y
        # TODO: convert this into torch code is possible
        self.length = [ np.sum(1 - np.equal(x, 0)) for x in X]
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        x_len = self.length[index]
        return x,y,x_len
    
    def __len__(self):
        return len(self.data)

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 60
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

train_dataset = MyData(input_tensor_train, target_tensor_train)
val_dataset = MyData(input_tensor_val, target_tensor_val)

dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE, 
                     drop_last=True,
                     shuffle=True)

val_dataset = DataLoader(val_dataset, batch_size = BATCH_SIZE, 
                     drop_last=True,
                     shuffle=False)

# Encoder Model

First we build a simple encoder model, which will be very similar to what you did in MP2. But instead of using a fully connected layer as the output, you should the return the output of your recurrent net (GRU/LSTM) as well as the hidden output. They are used in the decoder later.


In [0]:
## Feel free to change any parameters class definitions as long as you can change the training code, but make sure
## evaluation should get the tensor format it expects
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        ### TO - DO
        
    def forward(self, x, lens):
        '''
        Pseudo-code
        - Pass x through an embedding layer
        - Make sure x is correctly packed before the recurrent net 
        - Pass it through the recurrent net
        - Make sure the output is unpacked correctly
        - return output and hidden states from the recurrent net
        - Feel free to play around with dimensions - the training loop should help you determine the dimensions
        '''
        ### TO - DO
        
        return output, hidden


# Decoder Model
We will implement a Decoder model which uses an attention mechanism. We will implement the decoder as provided in https://arxiv.org/pdf/1409.0473.pdf. **Please read** the links provided above first, at the start of this assignment for review. The pseudo-code for your implementation should be somewhat as follows:



1.   The input is put through an encoder model which gives us the encoder output of shape *(batch_size, max_length, hidden_size)* and the encoder hidden state of shape *(batch_size, hidden_size)*. 
2.   Using the output your encoder you will calculate the score and subsequently the attention using following equations : 
<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg" alt="attention equation 0" width="800">
<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg" alt="attention equation 1" width="800">

3. Once you have calculated this attention vector, you pass the original input x through a embedding layer. The output of this embedding layer is concatenated with the attention vector which is passed into a GRU.

4. Finally you pass the output of the GRU into a fully connected layer with an output size same as that vocab, to see the probability of the most possible word.




In [0]:
## Feel free to change any parameters class definitions as long as you can change the training code, but make sure
## evaluation should get the tensor format it expects
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dec_units, enc_units, batch_sz):
        super(Decoder, self).__init__()
        ### TO - DO
    
    def forward(self, x, hidden, enc_output):
        '''
        Pseudo-code
        - Calculate the score using the formula shown above using encoder output and hidden output. 
        Note h_t is the hidden output of the decoder and h_s is the encoder output in the formula
        - Calculate the attention weights using softmax and 
        passing through V - which can be implemented as a fully connected layer
        - Finally find c_t which is a context vector where the shape of context_vector should be (batch_size, hidden_size)
        - You need to unsqueeze the context_vector for concatenating with x aas listed in Point 3 above
        - Pass this concatenated tensor to the GRU and follow as specified in Point 4 above

        Returns :
        output - shape = (batch_size, vocab)
        hidden state - shape = (batch_size, hidden size)
        '''
        ### TO - DO
        
        return x, state, attention_weights

In [0]:
### sort batch function to be able to use with pad_packed_sequence
def sort_batch(X, y, lengths):
    lengths, indx = lengths.sort(dim=0, descending=True)
    X = X[indx]
    y = y[indx]
    return X.transpose(0,1), y, lengths # transpose (batch x seq) to (seq x batch)

In [0]:
criterion = nn.CrossEntropyLoss()

def loss_function(real, pred):
    """ Only consider non-zero inputs in the loss; mask needed """
    #mask = 1 - np.equal(real, 0) # assign 0 to all above 0 and 1 to all 0s
    #print(mask)
    mask = real.ge(1).type(torch.cuda.FloatTensor)
    
    loss_ = criterion(pred, real) * mask 
    return torch.mean(loss_)

In [0]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## Feel free to change any parameters class definitions as long as you can change the training code, but make sure
## evaluation should get the tensor format it expects, this is only for reference
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, units, BATCH_SIZE)

encoder.to(device)
decoder.to(device)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), 
                       lr=0.001)

# Train your model

You will train your model here.
*   Pass the source sentence and their corresponding lengths into the encoder
*   Creating the decoder input using <start> tokens
*   Now we find out the decoder outputs conditioned on the previous predicted word usually, but in our training we use teacher forcing. Read more about teacher forcing at https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/



In [0]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    encoder.train()
    decoder.train()
    
    total_loss = 0
    
    for (batch, (inp, targ, inp_len)) in enumerate(dataset):
        loss = 0
        
        xs, ys, lens = sort_batch(inp, targ, inp_len)
        enc_output, enc_hidden = encoder(xs.to(device), lens, device)
        dec_hidden = enc_hidden
        
        # use teacher forcing - feeding the target as the next input (via dec_input)
        dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * BATCH_SIZE)
        
        # run code below for every timestep in the ys batch
        for t in range(1, ys.size(1)):
            predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                         dec_hidden.to(device), 
                                         enc_output.to(device))
            loss += loss_function(ys[:, t].to(device), predictions.to(device))
            #loss += loss_
            dec_input = ys[:, t].unsqueeze(1)
            
        
        batch_loss = (loss / int(ys.size(1)))
        total_loss += batch_loss
        
        optimizer.zero_grad()
        
        loss.backward()

        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.detach().item()))
        
        
    ### TODO: Save checkpoint for model
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    

# Evaluation


*   We evaluate on the test set.
*   In this evaluation, instead of using the concept of teacher forcing, we use the prediction of the decoder as the input to the decoder for the sequence of outputs.



In [0]:
start = time.time()

encoder.eval()
decoder.eval()

total_loss = 0

final_output = torch.zeros((len(target_tensor_val),max_length_tar))
target_output = torch.zeros((len(target_tensor_val),max_length_tar))

for (batch, (inp, targ, inp_len)) in enumerate(val_dataset):
    loss = 0
    xs, ys, lens = sort_batch(inp, targ, inp_len)
    enc_output, enc_hidden = encoder(xs.to(device), lens, device)
    dec_hidden = enc_hidden
    
    dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * BATCH_SIZE)
    curr_output = torch.zeros((ys.size(0), ys.size(1)))
    curr_output[:, 0] = dec_input.squeeze(1)

    for t in range(1, ys.size(1)): # run code below for every timestep in the ys batch
        predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                      dec_hidden.to(device), 
                                      enc_output.to(device))
        loss += loss_function(ys[:, t].to(device), predictions.to(device))
        dec_input = torch.argmax(predictions, dim=1).unsqueeze(1)
        curr_output[:, t] = dec_input.squeeze(1)
    final_output[batch*BATCH_SIZE:(batch+1)*BATCH_SIZE] = curr_output
    target_output[batch*BATCH_SIZE:(batch+1)*BATCH_SIZE] = targ
    batch_loss = (loss / int(ys.size(1)))
    total_loss += batch_loss

print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                    total_loss / N_BATCH))
print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

# Bleu Score Calculation for evaluation

Read more about Bleu Score at :


1.   https://en.wikipedia.org/wiki/BLEU
2.   https://www.aclweb.org/anthology/P02-1040.pdf

We expect your BLEU Scores to be in the range of for full credit. No partial credit :( 


*   BLEU-1 > 0.14
*   BLEU-2 > 0.08
*   BLEU-3 > 0.02
*   BLEU-4 > 0.15





In [0]:
def get_reference_candidate(target, pred):
  reference = list(target)
  reference = [targ_lang.idx2word[s] for s in np.array(reference[1:])]
  candidate = list(pred)
  candidate = [targ_lang.idx2word[s] for s in np.array(candidate[1:])]
  return reference, candidate

In [0]:
bleu_1 = 0.0
bleu_2 = 0.0
bleu_3 = 0.0
bleu_4 = 0.0
smoother = SmoothingFunction()
save_reference = []

for i in range(len(target_tensor_val)):
  reference, candidate = get_reference_candidate(target_output[i], final_output[i])
  #print(reference)
  #print(candidate)
  save_reference.append(reference)

  bleu_1 += sentence_bleu(reference, candidate, weights=(1, 0, 0, 0), smoothing_function=smoother.method1)
  bleu_2 += sentence_bleu(reference, candidate, weights=(0, 1, 0, 0), smoothing_function=smoother.method2)
  bleu_3 += sentence_bleu(reference, candidate, weights=(0, 0, 1, 0), smoothing_function=smoother.method3)
  bleu_4 += sentence_bleu(reference, candidate, weights=(0, 0, 0, 1), smoothing_function=smoother.method4)

print('Individual 1-gram: %f' % (bleu_1/len(target_tensor_val)))
print('Individual 2-gram: %f' % (bleu_2/len(target_tensor_val)))
print('Individual 3-gram: %f' % (bleu_3/len(target_tensor_val)))
print('Individual 4-gram: %f' % (bleu_4/len(target_tensor_val)))
assert(len(save_reference)==len(target_tensor_val))

# Save File for Submission
You just need to submit your **results.pickle** file to the autograder.

In [0]:
import pickle
from google.colab import drive
drive.mount('/content/drive')

In [0]:
with open('../drive/My Drive/results.pickle', 'wb') as fil:
    pickle.dump(save_candidate, fil)